In [133]:
import baostock as bs
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import pymongo
import json
from datetime import datetime
from Kline import DayKline,WeekKline,MonthKline

rsiOverBuy = 80
rsiOverSell = 20
rsiMiddle = 50

In [50]:
# dataList = []
# dayk = DayKline("sz.000001", "深发展")
# dayK2 = DayKline("sz.000002", "万科A")
# dataList.append(dayk.__dict__)
# dataList.append(dayK2.__dict__)
# dataListJson = json.dumps(dataList, ensure_ascii=False) 
# mydb = connectDB()
# collection = mydb["Kline_Day"]
# collection.insert_many(dataList)
bs.login()
rs = bs.query_stock_basic(code="sz.002030")
stockName = None
if (rs.error_code == '0') & rs.next():
    stockName = rs.get_row_data()[1]
print (stockName)
bs.logout()

login success!
达安基因
logout success!


In [28]:
dataListJson


'[{"code": "sz.000001", "name": "深发展"}, {"code": "sz.000002", "name": "万科A"}]'

In [137]:

##
# 连接数据库，这里用的mongoDB
#
def connectDB():
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient["tinyspark"]
    return mydb


##
#  下载日K线数据
#  code 股票代码
#  startDate 起始时间
#  endDate   结束时间
#
def downloadDailyStockKline(bsClient, stockCode, startdate, enddate):
    
    #查股票名字
    #返回示例数据
    #code	code_name	ipoDate	outDate	type	status
    #sh.600000	浦发银行	1999-11-10		1	1

    rs = bsClient.query_stock_basic(code=stockCode)
    stockName = None
    if (rs.error_code == '0') & rs.next():
        stockName = rs.get_row_data()[1]
    if stockName == None:
        raise RuntimeError("无此股票代码：", stockCode)
    
    #### 获取沪深A股历史K线数据 ####
    # 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。
    # 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
    rs = bsClient.query_history_k_data_plus(code,
        "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradeStatus,pctChg,isST",
        start_date=startdate, end_date=enddate,
        frequency="d", adjustflag="3")
    
    writeKlineToDb("day", rs)

#     df_status['close'] = df_status['close'].astype(float)
    
    return True

##
#  写数据库
#
#
def writeKlineToDb(period, resultSet):
    dataList = []
    while (resultSet.error_code == '0') & resultSet.next():
        # 获取一条记录，将记录合并在一起
#         data_list.append(rs.get_row_data())
        row = resultSet.get_row_data()
    
        kline = None
        recordDate = None
        
        
        if period == "day":
            kline = DayKline(row[1], stockName)
        elif period == "week":
            kline = WeekKline(row[1], stockName)
        elif period == "month":
            kline = MonthKline(row[1], stockName)
        elif period == "5m":
            kline = FiveMinKline(now[1], stockName)
        elif period == "30m":
            kline = ThirtyMinKline(now[1], stockName)
        elif period == "60m":
            kline = SixtyMinKline(now[1], stockName)
#         recordDate = datetime.strptime(row[0], "%Y-%m-%d")
        
        # 日K、月K、周K有专有属性
        if period == "day" or period == "week" or period == "month":
            recordDate = datetime.strptime(row[0], "%Y-%m-%d")
            kline.turn = row[10]
            kline.preClosePrice = row[6]
        else:
            recordDate = datetime.strptime(row[0], "%Y-%m-%dT%H:%M:%S.000Z")
        # 日K专有属性
        if period == "day":
            kline.tradeStatus = row[11]
            kline.changePercent = row[12]
            kline.isST = row[13]
        kline.date = recordDate
        kline.openPrice = row[2]
        kline.highPrice = row[3]
        kline.lowPrice = row[4]
        kline.closePrice = row[5]
        kline.volumn = row[7]
        kline.amount = row[8]
        kline.adjustflag = row[9]
        
        dataList.append(kline.__dict__)

#     dataListJson = json.dumps(dataList, ensure_ascii=False) 
    mydb = connectDB()
    collection = mydb[chooseCollection(period)]
    if len(dataList) > 0:
        collection.insert_many(dataList)
    else:
        raise RuntimeError("数据为空")

##
#  选择不同的Collection
#
def chooseCollection(period):
    periodCollection = {
        "day" : "Kline_Day",
        "week" : "Kline_Week",
        "month" : "Kline_Month",
        "5m" : "Kline_5m",
        "30m" : "Kline_30m",
        "60m" : "Kline_60m"
    }
    return periodCollection.get(period)
        
##
#  从数据库读取K线数据，转DataFrame
#  startDate、endDate在日线级别以上时，自动拼接成结束日23:59:59
#
def readStockKline(code, period, startDate, endDate):
    mydb = connectDB()
    cursor = None
    periodCollection = chooseCollection(period)
  
    if period == "day" or period == "week" or period == "month":
        startDate = datetime.strptime(startDate + "T00:00:00.000Z", "%Y-%m-%dT%H:%M:%S.000Z")
        endDate = datetime.strptime(endDate + "T23:59:59.000Z", "%Y-%m-%dT%H:%M:%S.000Z")
    
    
    cursor = mydb[periodCollection.get(period)].find({"code":code,"date":{"$gte":startDate, "$lte":endDate}})
    df =  pd.DataFrame(list(cursor))
    return df


def computeRSI(klineDataFrame):
    # 剔除停盘数据
    klineDataFrame = klineDataFrame[klineDataFrame['tradeStatus'] == '1']
    rsi_12days = ta.RSI(klineDataFrame['closePrice'],timeperiod=12)
    rsi_6days = ta.RSI(klineDataFrame['closePrice'],timeperiod=6)
    rsi_24days = ta.RSI(klineDataFrame['closePrice'],timeperiod=24)
    klineDataFrame['rsi_6days'] = rsi_6days
    klineDataFrame['rsi_12days'] = rsi_12days
    klineDataFrame['rsi_24days'] = rsi_24days
    ##添加参考线位置
    klineDataFrame['overBuy'] = rsiOverBuy
    klineDataFrame['overSell'] = rsiOverSell
    klineDataFrame['middle'] = rsiMiddle

    # RSI超卖和超买
    rsi_buy_position = klineDataFrame['rsi_6days'] > rsiOverBuy
    rsi_sell_position = klineDataFrame['rsi_6days'] < rsiOverSell
    klineDataFrame.loc[rsi_buy_position[(rsi_buy_position == True) & (rsi_buy_position.shift() == False)].index, '超买'] = '超买'
    klineDataFrame.loc[rsi_sell_position[(rsi_sell_position == True) & (rsi_sell_position.shift() == False)].index, '超卖'] = '超卖'
    return klineDataFrame

    

In [142]:
def downloadAllKlineDataOfSingleDay(date):
    bs.login()
    # 获取指定日期的指数、股票数据
    stock_rs = bs.query_all_stock(date)
    stock_df = stock_rs.get_data()
    data_df = pd.DataFrame()
    downloadedCount = 0
    for code in stock_df["code"]:
        downloadDailyStockKline(bs, code, date, date)
        downloadedCount = downloadedCount + 1
        if downloadedCount % 100 == 0 and downloadedCount > 0:
            print ("process:", downloadedCount, " of ", len(stock_df) )
#         k_rs = bs.query_history_k_data_plus(code, 
#             "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradeStatus,pctChg,isST",
#             date, date)
#         data_df = data_df.append(k_rs.get_data())
    bs.logout()


In [ ]:
downloadAllKlineDataOfSingleDay("2019-09-19")

login success!
downloaded: sh.000001
downloaded: sh.000002
downloaded: sh.000003
downloaded: sh.000004
downloaded: sh.000005
downloaded: sh.000006
downloaded: sh.000007
downloaded: sh.000008
downloaded: sh.000009
downloaded: sh.000010
process: 10  of  4200
downloaded: sh.000011
downloaded: sh.000012
downloaded: sh.000013
downloaded: sh.000015
downloaded: sh.000016
downloaded: sh.000017
downloaded: sh.000018
downloaded: sh.000019
downloaded: sh.000020
downloaded: sh.000021
process: 20  of  4200
downloaded: sh.000022
downloaded: sh.000025
downloaded: sh.000026
downloaded: sh.000027
downloaded: sh.000028
downloaded: sh.000029
downloaded: sh.000030
downloaded: sh.000031
downloaded: sh.000032
downloaded: sh.000033
process: 30  of  4200
downloaded: sh.000034
downloaded: sh.000035
downloaded: sh.000036
downloaded: sh.000037
downloaded: sh.000038
downloaded: sh.000039
downloaded: sh.000040
downloaded: sh.000041
downloaded: sh.000042
downloaded: sh.000043
process: 40  of  4200
downloaded: sh.00

downloaded: sh.600143
process: 340  of  4200
downloaded: sh.600145
downloaded: sh.600146
downloaded: sh.600148
downloaded: sh.600149
downloaded: sh.600150
downloaded: sh.600151
downloaded: sh.600152
downloaded: sh.600153
downloaded: sh.600155
downloaded: sh.600156
process: 350  of  4200
downloaded: sh.600157
downloaded: sh.600158
downloaded: sh.600159
downloaded: sh.600160
downloaded: sh.600161
downloaded: sh.600162
downloaded: sh.600163
downloaded: sh.600165
downloaded: sh.600166
downloaded: sh.600167
process: 360  of  4200
downloaded: sh.600168
downloaded: sh.600169
downloaded: sh.600170
downloaded: sh.600171
downloaded: sh.600172
downloaded: sh.600173
downloaded: sh.600175
downloaded: sh.600176
downloaded: sh.600177
downloaded: sh.600178
process: 370  of  4200
downloaded: sh.600179
downloaded: sh.600180
downloaded: sh.600182
downloaded: sh.600183
downloaded: sh.600184
downloaded: sh.600185
downloaded: sh.600186
downloaded: sh.600187
downloaded: sh.600188
downloaded: sh.600189
proces

downloaded: sh.600563
downloaded: sh.600565
process: 680  of  4200
downloaded: sh.600566
downloaded: sh.600567
downloaded: sh.600568
downloaded: sh.600569
downloaded: sh.600570
downloaded: sh.600571
downloaded: sh.600572
downloaded: sh.600573
downloaded: sh.600575
downloaded: sh.600576
process: 690  of  4200
downloaded: sh.600577
downloaded: sh.600578
downloaded: sh.600579
downloaded: sh.600580
downloaded: sh.600581
downloaded: sh.600582
downloaded: sh.600583
downloaded: sh.600584
downloaded: sh.600585
downloaded: sh.600586
process: 700  of  4200
downloaded: sh.600587
downloaded: sh.600588
downloaded: sh.600589
downloaded: sh.600590
downloaded: sh.600592
downloaded: sh.600593
downloaded: sh.600594
downloaded: sh.600595
downloaded: sh.600596
downloaded: sh.600597
process: 710  of  4200
downloaded: sh.600598
downloaded: sh.600599
downloaded: sh.600600
downloaded: sh.600601
downloaded: sh.600602
downloaded: sh.600603
downloaded: sh.600604
downloaded: sh.600605
downloaded: sh.600606
downlo

downloaded: sh.600981
downloaded: sh.600982
downloaded: sh.600983
downloaded: sh.600984
process: 1020  of  4200
downloaded: sh.600985
downloaded: sh.600986
downloaded: sh.600987
downloaded: sh.600988
downloaded: sh.600989
downloaded: sh.600990
downloaded: sh.600992
downloaded: sh.600993
downloaded: sh.600995
downloaded: sh.600996
process: 1030  of  4200
downloaded: sh.600997
downloaded: sh.600998
downloaded: sh.600999
downloaded: sh.601000
downloaded: sh.601001
downloaded: sh.601002
downloaded: sh.601003
downloaded: sh.601005
downloaded: sh.601006
downloaded: sh.601007
process: 1040  of  4200
downloaded: sh.601008
downloaded: sh.601009
downloaded: sh.601010
downloaded: sh.601011
downloaded: sh.601012
downloaded: sh.601015
downloaded: sh.601016
downloaded: sh.601018
downloaded: sh.601019
downloaded: sh.601020
process: 1050  of  4200
downloaded: sh.601021
downloaded: sh.601028
downloaded: sh.601038
downloaded: sh.601058
downloaded: sh.601066
downloaded: sh.601068
downloaded: sh.601069
do

In [138]:
if __name__ == '__main__':
    code = "sz.002030"
    startDate = "2019-08-01"
    endDate = "2019-09-20"
    ## 下载数据
    bs.login()
    result = downloadDailyStockKline(bs, code, startDate, endDate)
    bs.logout()
#     df.to_csv("/Users/matt/Documents/pythonworkspace/dailyKline.csv", encoding='utf_8_sig')
    
#     df = pd.read_csv("/Users/matt/Documents/pythonworkspace/dailyKline.csv")
    
    # 从数据库读数据
#     df = readStockKline(code, "day", startDate, endDate)
    
#     ## 计算开始
#     rsi = computeRSI(df)
#     dateArray = []
#     for rsiDate in rsi["date"]:
#         rsiDate = rsiDate.strftime("%Y-%m-%d")
#         dateArray.append(rsiDate)
#     rsi["date"] = dateArray
#     df2 = rsi[['date','rsi_6days','rsi_12days','rsi_24days','overBuy','overSell','middle']]
#     df2.index = rsi['date']
#     plot = df2.plot(title='RSI', figsize=(40,10))
#     fig = plot.get_figure() 
#     fig.savefig('/Users/matt/Downloads/rsi.png')
#     rsi.to_csv("/Users/matt/Documents/pythonworkspace/rsi.csv",encoding='utf_8_sig')
    

login success!
logout success!
